The geometries (e.g. shape files) for census tracts can be found at the following link: https://github.com/loganpowell/census-geojson/tree/master/GeoJSON/500k/2018. Each folder at that link represents a state, so all needed to be downloaded.

To download all of these, the following command was used:
svn checkout https://github.com/loganpowell/census-geojson/trunk/GeoJSON/500k/2018

This script prepares all tracts in the 2018 folder as a single geoJSON file. That file is compressed to fit within the github repo.

In [1]:
import os
import json
import gzip
from urllib.request import urlopen
import pandas as pd
import geopandas as gp
import plotly.graph_objects as go

In [2]:
tract_all_path = 'all_census_tract_shapes.json.gz'

if not os.path.exists(tract_all_path):
    # Read all census tract geography files, store each in a list
    ts_json = []
    for item in os.listdir('./2018'):
        subfolder = os.path.join('./2018', item)
        if not os.path.isdir(subfolder) or item == '.svn':
            continue
        with open(os.path.join(subfolder, 'tract.json')) as f:
            ts_json.append(json.load(f))

    # combine all features from all items in that list
    TRACT_ALL = ts_json[0]
    print(len(TRACT_ALL['features']))
    for tract_ in ts_json[1:]:
        TRACT_ALL['features'] += tract_['features']
    print(len(TRACT_ALL['features']))

    # create an id based on the geoid
    for i, feature in enumerate(TRACT_ALL['features']):
        TRACT_ALL['features'][i]['id'] = feature['properties']['GEOID']

    # write to file
    with gzip.GzipFile(tract_all_path, 'w') as f:
        f.write(json.dumps(TRACT_ALL).encode('utf-8'))
else:
    with gzip.GzipFile(tract_all_path, 'r') as f:
        TRACT_ALL = json.loads(f.read().decode('utf-8'))

In [3]:
TRACT_ALL['features'][0]

{'type': 'Feature',
 'geometry': {'bbox': [-93.164718,
   30.190175999999994,
   -93.128872,
   30.227656000000003],
  'type': 'Polygon',
  'coordinates': [[[-93.16467999999999, 30.216629],
    [-93.163923, 30.21663],
    [-93.160507, 30.21945],
    [-93.154901, 30.224311],
    [-93.15481899999999, 30.227639999999997],
    [-93.147663, 30.227656000000003],
    [-93.128895, 30.199903000000003],
    [-93.128875, 30.191060999999994],
    [-93.128872, 30.190565],
    [-93.13452799999999, 30.190175999999994],
    [-93.13822499999999, 30.190919],
    [-93.142887, 30.193118999999996],
    [-93.145749, 30.194968],
    [-93.149637, 30.198355],
    [-93.141076, 30.198414],
    [-93.140619, 30.198428000000003],
    [-93.142685, 30.202377000000002],
    [-93.14484399999999, 30.202927999999996],
    [-93.144235, 30.205507999999995],
    [-93.14268899999999, 30.206107],
    [-93.14846299999999, 30.215990999999995],
    [-93.154721, 30.213483000000004],
    [-93.15996299999999, 30.213910999999996],
 

---

This shows how a geopandas dataframe can be prepared from the geoJSON.

In [4]:
#gpdf = gp.read_file(tract_all_path)
gpdf = gp.GeoDataFrame.from_features(TRACT_ALL['features'])
gpdf

,geometry,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,"POLYGON ((-93.16468 30.21663, -93.16392 30.216...",22,019,980000,1400000US22019980000,22019980000,9800,CT,5398742,2339
1,"POLYGON ((-93.04200 32.77714, -93.04118 32.777...",22,027,950100,1400000US22027950100,22027950100,9501,CT,707275075,30909943
2,"POLYGON ((-93.98127 31.93953, -93.97938 31.942...",22,031,950700,1400000US22031950700,22031950700,9507,CT,601224947,16024333
3,"POLYGON ((-91.15304 30.49978, -91.14977 30.509...",22,033,000200,1400000US22033000200,22033000200,2,CT,3893646,0
4,"POLYGON ((-91.18012 30.43970, -91.17940 30.439...",22,033,002200,1400000US22033002200,22033002200,22,CT,939392,0
...,...,...,...,...,...,...,...,...,...,...
73869,"POLYGON ((-116.77682 47.67338, -116.76478 47.6...",16,055,001600,1400000US16055001600,16055001600,16,CT,2178716,959415
73870,"POLYGON ((-116.98078 46.40523, -116.97863 46.4...",16,069,961000,1400000US16069961000,16069961000,9610,CT,17133787,0
73871,"POLYGON ((-111.98391 43.55360, -111.97477 43.5...",16,019,970402,1400000US16019970402,16019970402,9704.02,CT,35867172,0
73872,"POLYGON ((-112.69692 43.79789, -112.69497 43.7...",16,051,960100,1400000US16051960100,16051960100,9601,CT,2475576821,22042207
